# Introduction
To do downstream analysis later in the project we need:
1. Geolocation of where the sample was taken
2. Source that the sample was isolated from
3. Date on which the sample was collected
4. Filter out lab strains

None of these things have a column in any of the tables from the NCBI database. See this [paper](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6380228/) that describes how metadata in NCBI sucks.
 
However, there is a column called 'sample_attribute' in the SRA and Sample table where a submitter can add additional information about a sample. As 'sample_attribute' does not require a specific format or specific information. The information found there varies greatly between samples. Some organizations (rivm) that submit data to the NCBI have a consisted format for this column which then also varies per organization, others do not. This makes it very challenging to extrapolate the information mentioned above for all samples. In this notebook we attempt to extract this information.

In [1]:
import pandas as pd
import re
import pyarrow.feather as feather
from collections import defaultdict

Functions written for this notebook are stored in wrangling_funcs.py. Please look there for documentation and tests.

In [2]:
import wrangling_funcs

# Reading in the data
---

R has a nice package called SRAdb that you can use to query the NCBI database. However, I prefer working in Python. So we are querying the data in R using SRAdb and then exporting it in feather format for use here. There might be a way to directly get a dump of the SRA database and query it without using SRAdb. I will look into this.

The default index of a dataframe is not useful to us. Instead, we use the run_accession, these should be unique. This way we can keep track when we split the metadata into a separate dataframe.

In [3]:
file_path = '../../results/SRA.feather'
data = feather.read_feather(file_path)
# data = feather.read_feather(snakemake.input[0])

metadata_df = pd.DataFrame(data)
metadata_df = metadata_df.convert_dtypes()
metadata_df.set_index('run_accession', inplace=True)

print(f'---Number of rows: {metadata_df.shape[0]}, Number of columns: {metadata_df.shape[1]}---')
metadata_df.head()

---Number of rows: 45124, Number of columns: 52---


,sra_ID,run_ID,run_alias,run_date,updated_date,spots,bases,run_center,experiment_name,run_attribute,...,center_project_name,study_description,study_url_link,study_attribute,submission_ID,submission_accession,submission_center,submission_lab,sradb_updated,scientific_name
run_accession,,,,,,,,,,,,,,,,,,,,,
DRR002036,1868,1872,DRR002036,2010-07-13,2023-06-10,2648846,529769200,NANKAI,DRX001488,<NA>,...,Escherichia coli,<NA>,<NA>,<NA>,496,DRA000563,NANKAI,TEDA School of Biological Sciences and Biotech...,2023-12-09 17:25:46,Escherichia coli
DRR002039,1869,1904,DRR002039,2011-07-18,2023-06-16,3448927,689785400,NANKAI,DRX001491,<NA>,...,Escherichia coli,<NA>,<NA>,<NA>,496,DRA000563,NANKAI,TEDA School of Biological Sciences and Biotech...,2023-12-09 17:25:46,Escherichia coli
DRR002034,1870,1896,DRR002034,2011-07-18,2023-06-16,3456264,691252800,NANKAI,DRX001486,<NA>,...,Escherichia coli,<NA>,<NA>,<NA>,496,DRA000563,NANKAI,TEDA School of Biological Sciences and Biotech...,2023-12-09 17:25:46,Escherichia coli
DRR002025,1871,1884,DRR002025,2011-07-18,2023-06-03,4686868,937373600,NANKAI,DRX001477,<NA>,...,Escherichia coli,<NA>,<NA>,<NA>,496,DRA000563,NANKAI,TEDA School of Biological Sciences and Biotech...,2023-12-09 17:25:46,Escherichia coli
DRR002037,1872,1886,DRR002037,2010-07-13,2023-06-02,3039542,607908400,NANKAI,DRX001489,<NA>,...,Escherichia coli,<NA>,<NA>,<NA>,496,DRA000563,NANKAI,TEDA School of Biological Sciences and Biotech...,2023-12-09 17:25:46,Escherichia coli


# Finding metadata in the sample_attribute
---

All the metadata we are interested in is contained in the 'sample_attribute' column. From what we could see most of the information in this column is split by '||' characters. The information between these characters is then often split using ':'. We will use this to make key value pairs which we will then turn into a dataframe.

In [4]:
sample_attribute = metadata_df['sample_attribute']
faulty_lines = []
correct_lines = []

pattern = re.compile(r"^[mM]is{1,3}ing$|^[nN]ot.*|^[oO]ther$|^[uU]nspecified$|^\.$|^\*$|\?|^[Nn]a[nN]$|^[Nn]a$|^ $|^[Uu]nknown$|^[Nn]o$")

for line, identity in zip(sample_attribute, sample_attribute.index):
    line = line.split("||")
    line_items = defaultdict(list)
    for subitem in line:
        try:
            key, value = subitem.split(': ', 1)
            strip_key = wrangling_funcs.clean_string(key)
            strip_value = value.strip()
            if pattern.match(strip_value):
                strip_value = pd.NA
            line_items[strip_key] = strip_value
            line_items['run_accession'] = identity
        except ValueError:
            faulty_lines.append((identity, line))
    correct_lines.append(line_items)

smpl_att_df = pd.DataFrame(correct_lines)
smpl_att_df = smpl_att_df.convert_dtypes()
smpl_att_df.set_index('run_accession', inplace=True)

print(f'---Number of rows: {smpl_att_df.shape[0]}, Number of columns: {smpl_att_df.shape[1]}---')
smpl_att_df

---Number of rows: 45124, Number of columns: 156---


,sample_name,strain,sample_comment,collection_date,env_broad_scale,env_local_scale,env_medium,geo_loc_name,isol_growth_condt,lat_lon,...,protocol,arrayexpress_strainorline,arrayexpress_phenotype,lab_host,instrument_model,identified_by,isolation_site,isolation_hospital,week_year_isolated,plant_associated_environmental_package
run_accession,,,,,,,,,,,,,,,,,,,,,
DRR002036,DRS001446,5W19,Replicate evolution using glucose-limited chem...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,NaN,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,<NA>
DRR002039,DRS001449,8W17,Replicate evolution using glucose-limited chem...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,NaN,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,<NA>
DRR002034,DRS001444,5W3,Replicate evolution using glucose-limited chem...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,NaN,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,<NA>
DRR002025,DRS001435,3X31,Replicate evolution using glucose-limited chem...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,NaN,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,<NA>
DRR002037,DRS001447,5W21,Replicate evolution using glucose-limited chem...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,NaN,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ERR9079443,SAL_IB8517AA,YA00197045,<NA>,2020-02-07,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,NaN,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,<NA>
ERR9079436,SAL_IB8513AA,YA00230774,<NA>,2020-12-23,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,NaN,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,<NA>
ERR9079429,SAL_IB8509AA,YA00230709,<NA>,2020-12-15,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,NaN,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,<NA>


### Searching for geographic data

There is no consistent column that contains the geolocation. To (hopefully) obtain the geolocation we use regex to find keywords in the column names of the dataframe. The matched columns are then combined in a single column while handling NaN values.

In [5]:
geo_col_matches = wrangling_funcs.find_columns(['geo', 'geographic', 'country', 'continent'], smpl_att_df, ['longitude', 'latitude', 'depth'])
print(f'The following columns matched the keywords: {geo_col_matches}')

smpl_att_df = wrangling_funcs.combine_columns(smpl_att_df, list(geo_col_matches), 'inferred_location')
smpl_att_df.drop(geo_col_matches, inplace=True, axis=1)

smpl_att_df['inferred_continent'], smpl_att_df['inferred_country'], smpl_att_df['inferred_city'] = zip(*smpl_att_df['inferred_location'].map(wrangling_funcs.clean_geo))
smpl_att_df.drop('inferred_location', axis=1, inplace=True)

smpl_att_df = smpl_att_df.convert_dtypes()

print(f'---Number of rows: {smpl_att_df.shape[0]}, Number of columns: {smpl_att_df.shape[1]}---')
smpl_att_df.head()

The following columns matched the keywords: {'isolation_country', 'country', 'geographic_location_altitude', 'continent', 'geographic_location', 'geographic_location_region_and_locality', 'geographic_location_country_and_or_sea', 'geo_loc_name'}
ERROR: Country name not recognized by Pycountry. Continent set to NaN!
---Number of rows: 45124, Number of columns: 151---


,sample_name,strain,sample_comment,collection_date,env_broad_scale,env_local_scale,env_medium,isol_growth_condt,lat_lon,num_replicons,...,lab_host,instrument_model,identified_by,isolation_site,isolation_hospital,week_year_isolated,plant_associated_environmental_package,inferred_continent,inferred_country,inferred_city
run_accession,,,,,,,,,,,,,,,,,,,,,
DRR002036,DRS001446,5W19,Replicate evolution using glucose-limited chem...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
DRR002039,DRS001449,8W17,Replicate evolution using glucose-limited chem...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
DRR002034,DRS001444,5W3,Replicate evolution using glucose-limited chem...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
DRR002025,DRS001435,3X31,Replicate evolution using glucose-limited chem...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
DRR002037,DRS001447,5W21,Replicate evolution using glucose-limited chem...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


### Searching for the sample collection data

There is no consistent column that contains the date. To (hopefully) obtain the date we use regex to find keywords in the column names of the dataframe. The matched columns are then combined in a single column while handling NaN values.

In [6]:
date_col_matches = wrangling_funcs.find_columns(['date', 'year'], smpl_att_df, ['update'])
print(f'The following columns matched the keywords: {date_col_matches}')
smpl_att_df = wrangling_funcs.combine_columns(smpl_att_df, list(date_col_matches), 'inferred_collection_year')
smpl_att_df.drop(date_col_matches, inplace=True, axis=1)


date = smpl_att_df['inferred_collection_year'].str.extract(r'^(\d{4})', expand=False) # Extract the year
smpl_att_df['inferred_collection_year'] = pd.to_numeric(date) # cast year to int

smpl_att_df = smpl_att_df.convert_dtypes()
print(f'---Number of rows: {smpl_att_df.shape[0]}, Number of columns: {smpl_att_df.shape[1]}---')
smpl_att_df.head()

The following columns matched the keywords: {'year_isolated', 'week_year_isolated', 'isolation_date', 'year', 'collection_date', 'receipt_date'}
---Number of rows: 45124, Number of columns: 146---


,sample_name,strain,sample_comment,env_broad_scale,env_local_scale,env_medium,isol_growth_condt,lat_lon,num_replicons,project_name,...,lab_host,instrument_model,identified_by,isolation_site,isolation_hospital,plant_associated_environmental_package,inferred_continent,inferred_country,inferred_city,inferred_collection_year
run_accession,,,,,,,,,,,,,,,,,,,,,
DRR002036,DRS001446,5W19,Replicate evolution using glucose-limited chem...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
DRR002039,DRS001449,8W17,Replicate evolution using glucose-limited chem...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
DRR002034,DRS001444,5W3,Replicate evolution using glucose-limited chem...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
DRR002025,DRS001435,3X31,Replicate evolution using glucose-limited chem...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
DRR002037,DRS001447,5W21,Replicate evolution using glucose-limited chem...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


### Searching for sample isolation source
There is no consistent column that contains the isolation source. To (hopefully) obtain the isolation source we use regex to find keywords in the column names of the dataframe. The matched columns are then combined in a single column while handling NaN values.


In [7]:
isolate_matches = wrangling_funcs.find_columns(['sample', 'source', 'environment', 'env', 'site'], smpl_att_df, ['name', 'provider', 'comment'])
print(isolate_matches)

smpl_att_df = wrangling_funcs.combine_columns(smpl_att_df, list(isolate_matches), "inferred_source")
smpl_att_df.drop(isolate_matches, inplace=True, axis=1)

smpl_att_df['inferred_source'] = smpl_att_df['inferred_source'].apply(wrangling_funcs.clean_source)

smpl_att_df = smpl_att_df.convert_dtypes()
print(f'---Number of rows: {smpl_att_df.shape[0]}, Number of columns: {smpl_att_df.shape[1]}---')
smpl_att_df.head()

{'miscellaneous_environmental_package', 'isolation_source_non_host_associated', 'plant_associated_environmental_package', 'biosamplemodel', 'environmental_medium', 'sample_type', 'source_subgroup', 'broad_scale_environmental_context', 'sample', 'human_associated_environmental_package', 'source_group', 'env_medium', 'environment_material', 'source', 'env_local_scale', 'environmental_sample_type', 'wastewater_sludge_environmental_package', 'sample_description', 'env_broad_scale', 'host_associated_environmental_package', 'environment_biome', 'water_environmental_package', 'environment_feature', 'isolation_source_host_associated', 'isolation_site', 'environmental_sample', 'isolation_source', 'sample_source', 'source_site', 'local_environmental_context'}
---Number of rows: 45124, Number of columns: 117---


,sample_name,strain,sample_comment,isol_growth_condt,lat_lon,num_replicons,project_name,ref_biomaterial,host,locus_tag_prefix,...,arrayexpress_phenotype,lab_host,instrument_model,identified_by,isolation_hospital,inferred_continent,inferred_country,inferred_city,inferred_collection_year,inferred_source
run_accession,,,,,,,,,,,,,,,,,,,,,
DRR002036,DRS001446,5W19,Replicate evolution using glucose-limited chem...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,NaN,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
DRR002039,DRS001449,8W17,Replicate evolution using glucose-limited chem...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,NaN,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
DRR002034,DRS001444,5W3,Replicate evolution using glucose-limited chem...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,NaN,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
DRR002025,DRS001435,3X31,Replicate evolution using glucose-limited chem...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,NaN,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
DRR002037,DRS001447,5W21,Replicate evolution using glucose-limited chem...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,NaN,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


### First round removal of lab strains
We don't want sample from lab strains, so we filter out rows based on some keywords

In [8]:
keywords_to_exclude = ["replicate", "mutant"]
smpl_att_df = smpl_att_df[~smpl_att_df.sample_comment.str.contains('|'.join(keywords_to_exclude), case=False, na=False)]
smpl_att_df = smpl_att_df[~smpl_att_df.genotype.str.contains('|'.join(keywords_to_exclude), case=False, na=False)]
smpl_att_df = smpl_att_df[~smpl_att_df.lab_experiment_type.str.contains('|'.join(keywords_to_exclude), case=False, na=False)]

print(f'---Number of rows: {smpl_att_df.shape[0]}, Number of columns: {smpl_att_df.shape[1]}---')
smpl_att_df.head()

---Number of rows: 45078, Number of columns: 117---


,sample_name,strain,sample_comment,isol_growth_condt,lat_lon,num_replicons,project_name,ref_biomaterial,host,locus_tag_prefix,...,arrayexpress_phenotype,lab_host,instrument_model,identified_by,isolation_hospital,inferred_continent,inferred_country,inferred_city,inferred_collection_year,inferred_source
run_accession,,,,,,,,,,,,,,,,,,,,,
DRR015582,DRS013790,<NA>,"Genome of Klebsiella oxytoca, GTC_14652, Gifu0...",<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,NaN,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
DRR015581,DRS013790,<NA>,"Genome of Klebsiella oxytoca, GTC_14652, Gifu0...",<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,NaN,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
DRR015585,DRS013792,<NA>,"Genome of Klebsiella pneumoniae, GTC_11259, Gi...",<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,NaN,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
DRR015586,DRS013792,<NA>,"Genome of Klebsiella pneumoniae, GTC_11259, Gi...",<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,NaN,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
DRR015588,DRS013793,<NA>,"Genome of Kluyvera intermedia, GTC_01506T, Gif...",<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,NaN,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


### Remove non relevant columns
We have a ton of columns and very few of them are actually usefull to us. Let's remove all not relevant columns

In [9]:
smpl_att_df = smpl_att_df[['strain', 'inferred_collection_year', 'inferred_source', 'inferred_continent', 'inferred_country', 'inferred_city', 'geographic_location_latitude', 'geographic_location_longitude']]

print(f'---Number of rows: {smpl_att_df.shape[0]}, Number of columns: {smpl_att_df.shape[1]}---')
smpl_att_df.head()

---Number of rows: 45078, Number of columns: 8---


,strain,inferred_collection_year,inferred_source,inferred_continent,inferred_country,inferred_city,geographic_location_latitude,geographic_location_longitude
run_accession,,,,,,,,
DRR015582,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
DRR015581,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
DRR015585,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
DRR015586,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
DRR015588,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


# Combine sample_attribute metadata with rest of the data
---
Now that we have extracted the metadata that we wanted we can combine it back to the original dataframe. We only want to keep rows that have values for the collection_year/source/country because we require this downstream.

In [10]:
combined_df = metadata_df.join(smpl_att_df)
cols = ['inferred_collection_year', 'inferred_source', 'inferred_country']
combined_df = combined_df.dropna(subset=cols)

combined_df = combined_df.convert_dtypes()
print(f'---Number of rows: {combined_df.shape[0]}, Number of columns: {combined_df.shape[1]}---')
combined_df.head()

---Number of rows: 24971, Number of columns: 60---


,sra_ID,run_ID,run_alias,run_date,updated_date,spots,bases,run_center,experiment_name,run_attribute,...,sradb_updated,scientific_name,strain,inferred_collection_year,inferred_source,inferred_continent,inferred_country,inferred_city,geographic_location_latitude,geographic_location_longitude
run_accession,,,,,,,,,,,,,,,,,,,,,
DRR065950,60940,60941,DRR065950,<NA>,2023-06-18,163482,2261498784,<NA>,DRX060093,<NA>,...,2023-12-09 17:25:46,Escherichia coli,MRY15-117,2012,MIGS.ba,Asia,Japan,<NA>,<NA>,<NA>
DRR065949,60942,60940,DRR065949,<NA>,2023-06-20,163482,1515099158,<NA>,DRX060092,<NA>,...,2023-12-09 17:25:46,Escherichia coli,20Ec-P-124,2008,MIGS.ba,Asia,Japan,<NA>,<NA>,<NA>
DRR065951,60944,60943,DRR065951,<NA>,2023-06-21,163482,1398527654,<NA>,DRX060094,<NA>,...,2023-12-09 17:25:46,Escherichia coli,MRY15-131,2013,MIGS.ba,Asia,Japan,<NA>,<NA>,<NA>
DRR067778,62654,62654,DRR067778,<NA>,2023-06-20,35065,247198598,<NA>,DRX061698,<NA>,...,2023-12-09 17:25:46,Klebsiella pneumoniae,KP01,2014,"rectal swab,MIGS.ba.human-associated",Asia,Thailand,Bangkok,<NA>,<NA>
DRR070681,65464,65468,DRR070681,<NA>,2023-06-18,69700,383913853,<NA>,DRX064631,<NA>,...,2023-12-09 17:25:46,Escherichia coli,M105,2015,"MIGS.ba.human-associated,human blood",Asia,Myanmar,Yangon,<NA>,<NA>


### Remove lab strain samples from combined dataframe
After combining the dataframes we have new columns that might give us more information if a particular run comes from a lab strain, so we filter again.


In [11]:
keywords_to_exclude = ["replicate", "mutant"]

combined_df = combined_df[~combined_df.description.str.contains('|'.join(keywords_to_exclude), case=False, na=False)]
combined_df = combined_df[~combined_df.study_title.str.contains('|'.join(keywords_to_exclude), case=False, na=False)]
combined_df = combined_df[~combined_df.study_abstract.str.contains('|'.join(keywords_to_exclude), case=False, na=False)]
combined_df = combined_df[~combined_df.study_description.str.contains('|'.join(keywords_to_exclude), case=False, na=False)]

print(f'---Number of rows: {combined_df.shape[0]}, Number of columns: {combined_df.shape[1]}---')
combined_df.head()

---Number of rows: 24969, Number of columns: 60---


,sra_ID,run_ID,run_alias,run_date,updated_date,spots,bases,run_center,experiment_name,run_attribute,...,sradb_updated,scientific_name,strain,inferred_collection_year,inferred_source,inferred_continent,inferred_country,inferred_city,geographic_location_latitude,geographic_location_longitude
run_accession,,,,,,,,,,,,,,,,,,,,,
DRR065950,60940,60941,DRR065950,<NA>,2023-06-18,163482,2261498784,<NA>,DRX060093,<NA>,...,2023-12-09 17:25:46,Escherichia coli,MRY15-117,2012,MIGS.ba,Asia,Japan,<NA>,<NA>,<NA>
DRR065949,60942,60940,DRR065949,<NA>,2023-06-20,163482,1515099158,<NA>,DRX060092,<NA>,...,2023-12-09 17:25:46,Escherichia coli,20Ec-P-124,2008,MIGS.ba,Asia,Japan,<NA>,<NA>,<NA>
DRR065951,60944,60943,DRR065951,<NA>,2023-06-21,163482,1398527654,<NA>,DRX060094,<NA>,...,2023-12-09 17:25:46,Escherichia coli,MRY15-131,2013,MIGS.ba,Asia,Japan,<NA>,<NA>,<NA>
DRR067778,62654,62654,DRR067778,<NA>,2023-06-20,35065,247198598,<NA>,DRX061698,<NA>,...,2023-12-09 17:25:46,Klebsiella pneumoniae,KP01,2014,"rectal swab,MIGS.ba.human-associated",Asia,Thailand,Bangkok,<NA>,<NA>
DRR070681,65464,65468,DRR070681,<NA>,2023-06-18,69700,383913853,<NA>,DRX064631,<NA>,...,2023-12-09 17:25:46,Escherichia coli,M105,2015,"MIGS.ba.human-associated,human blood",Asia,Myanmar,Yangon,<NA>,<NA>


### Remove samples submitted by the RIVM
Since we will already have the RIVM samples in the database there is no need to have them here.

In [12]:
combined_df = combined_df[~combined_df.submission_center.str.contains("RIVM", na=False)] # Remove samples submitted by the RIVM

print(f'---Number of rows: {combined_df.shape[0]}, Number of columns: {combined_df.shape[1]}---')
combined_df.head()

---Number of rows: 24343, Number of columns: 60---


,sra_ID,run_ID,run_alias,run_date,updated_date,spots,bases,run_center,experiment_name,run_attribute,...,sradb_updated,scientific_name,strain,inferred_collection_year,inferred_source,inferred_continent,inferred_country,inferred_city,geographic_location_latitude,geographic_location_longitude
run_accession,,,,,,,,,,,,,,,,,,,,,
DRR065950,60940,60941,DRR065950,<NA>,2023-06-18,163482,2261498784,<NA>,DRX060093,<NA>,...,2023-12-09 17:25:46,Escherichia coli,MRY15-117,2012,MIGS.ba,Asia,Japan,<NA>,<NA>,<NA>
DRR065949,60942,60940,DRR065949,<NA>,2023-06-20,163482,1515099158,<NA>,DRX060092,<NA>,...,2023-12-09 17:25:46,Escherichia coli,20Ec-P-124,2008,MIGS.ba,Asia,Japan,<NA>,<NA>,<NA>
DRR065951,60944,60943,DRR065951,<NA>,2023-06-21,163482,1398527654,<NA>,DRX060094,<NA>,...,2023-12-09 17:25:46,Escherichia coli,MRY15-131,2013,MIGS.ba,Asia,Japan,<NA>,<NA>,<NA>
DRR067778,62654,62654,DRR067778,<NA>,2023-06-20,35065,247198598,<NA>,DRX061698,<NA>,...,2023-12-09 17:25:46,Klebsiella pneumoniae,KP01,2014,"rectal swab,MIGS.ba.human-associated",Asia,Thailand,Bangkok,<NA>,<NA>
DRR070681,65464,65468,DRR070681,<NA>,2023-06-18,69700,383913853,<NA>,DRX064631,<NA>,...,2023-12-09 17:25:46,Escherichia coli,M105,2015,"MIGS.ba.human-associated,human blood",Asia,Myanmar,Yangon,<NA>,<NA>


### Replace missing value synonyms with NA
Once again after combining and obtaining new columns there are values in the rows that indicate NaNs

In [14]:
na_synonyms = {r'^\*$', r'^-$', r'^\.$', r'^[Nn]one$', r'^[Nn]an$', r'^[Uu]nknown$', r'(?i)^not[ _-]collected$', r'(?i)^not[ _-]provided', r'^\?$', r'^ $', r'(?i)^not[ _-]applicable$', r'^[Nn]a$', r'^[Nn]o$', r'^[Oo]ther$', r'^[Mm]is{1,3}ing$', r'^[Uu]nspecified$', r'^[Nn]ot[ ]available$', r'^[Nn]ot[ :]available[:] not collected$', '^[Nn]ot[ :]available[:] to be reported later$'}

combined_df = combined_df.replace(to_replace=na_synonyms, value=pd.NA, regex=True)

print(f'---Number of rows: {combined_df.shape[0]}, Number of columns: {combined_df.shape[1]}---')
combined_df.head()

---Number of rows: 24343, Number of columns: 60---


,sra_ID,run_ID,run_alias,run_date,updated_date,spots,bases,run_center,experiment_name,run_attribute,...,sradb_updated,scientific_name,strain,inferred_collection_year,inferred_source,inferred_continent,inferred_country,inferred_city,geographic_location_latitude,geographic_location_longitude
run_accession,,,,,,,,,,,,,,,,,,,,,
DRR065950,60940,60941,DRR065950,<NA>,2023-06-18,163482,2261498784,<NA>,DRX060093,<NA>,...,2023-12-09 17:25:46,Escherichia coli,MRY15-117,2012,MIGS.ba,Asia,Japan,<NA>,<NA>,<NA>
DRR065949,60942,60940,DRR065949,<NA>,2023-06-20,163482,1515099158,<NA>,DRX060092,<NA>,...,2023-12-09 17:25:46,Escherichia coli,20Ec-P-124,2008,MIGS.ba,Asia,Japan,<NA>,<NA>,<NA>
DRR065951,60944,60943,DRR065951,<NA>,2023-06-21,163482,1398527654,<NA>,DRX060094,<NA>,...,2023-12-09 17:25:46,Escherichia coli,MRY15-131,2013,MIGS.ba,Asia,Japan,<NA>,<NA>,<NA>
DRR067778,62654,62654,DRR067778,<NA>,2023-06-20,35065,247198598,<NA>,DRX061698,<NA>,...,2023-12-09 17:25:46,Klebsiella pneumoniae,KP01,2014,"rectal swab,MIGS.ba.human-associated",Asia,Thailand,Bangkok,<NA>,<NA>
DRR070681,65464,65468,DRR070681,<NA>,2023-06-18,69700,383913853,<NA>,DRX064631,<NA>,...,2023-12-09 17:25:46,Escherichia coli,M105,2015,"MIGS.ba.human-associated,human blood",Asia,Myanmar,Yangon,<NA>,<NA>


### Throw away empty columns
We want to filter out columns that only have NaN values so there is less cluter

In [16]:
combined_df = combined_df.dropna(axis=1, how='all')

print(f'---Number of rows: {combined_df.shape[0]}, Number of columns: {combined_df.shape[1]}---')
combined_df.head()

---Number of rows: 24343, Number of columns: 56---


,sra_ID,run_ID,run_alias,updated_date,spots,bases,run_center,experiment_name,run_attribute,experiment_ID,...,sradb_updated,scientific_name,strain,inferred_collection_year,inferred_source,inferred_continent,inferred_country,inferred_city,geographic_location_latitude,geographic_location_longitude
run_accession,,,,,,,,,,,,,,,,,,,,,
DRR065950,60940,60941,DRR065950,2023-06-18,163482,2261498784,<NA>,DRX060093,<NA>,55877,...,2023-12-09 17:25:46,Escherichia coli,MRY15-117,2012,MIGS.ba,Asia,Japan,<NA>,<NA>,<NA>
DRR065949,60942,60940,DRR065949,2023-06-20,163482,1515099158,<NA>,DRX060092,<NA>,55875,...,2023-12-09 17:25:46,Escherichia coli,20Ec-P-124,2008,MIGS.ba,Asia,Japan,<NA>,<NA>,<NA>
DRR065951,60944,60943,DRR065951,2023-06-21,163482,1398527654,<NA>,DRX060094,<NA>,55874,...,2023-12-09 17:25:46,Escherichia coli,MRY15-131,2013,MIGS.ba,Asia,Japan,<NA>,<NA>,<NA>
DRR067778,62654,62654,DRR067778,2023-06-20,35065,247198598,<NA>,DRX061698,<NA>,57390,...,2023-12-09 17:25:46,Klebsiella pneumoniae,KP01,2014,"rectal swab,MIGS.ba.human-associated",Asia,Thailand,Bangkok,<NA>,<NA>
DRR070681,65464,65468,DRR070681,2023-06-18,69700,383913853,<NA>,DRX064631,<NA>,60206,...,2023-12-09 17:25:46,Escherichia coli,M105,2015,"MIGS.ba.human-associated,human blood",Asia,Myanmar,Yangon,<NA>,<NA>


## Make a clean TSV using a selection of columns for parsing with bash when downloading through SRAtools

In [21]:
clean_tsv = combined_df['platform']
clean_tsv.head()

run_accession
DRR065950     PACBIO_SMRT
DRR065949     PACBIO_SMRT
DRR065951     PACBIO_SMRT
DRR067778     PACBIO_SMRT
DRR070681     PACBIO_SMRT
                 ...     
ERR9079443       ILLUMINA
ERR9079436       ILLUMINA
ERR9079429       ILLUMINA
ERR9079439       ILLUMINA
ERR9079431       ILLUMINA
Name: platform, Length: 24343, dtype: string

## Write out files

In [ ]:
clean_tsv.to_csv(snakemake.output[1], sep='\t')
combined_df.to_csv(snakemake.output[0], na_rep='NaN')